(569, 9)

In [21]:
## 5 datasets for 5 personality traits

df_A = df[['scene_id', 'text', 'character', 'cAGR']]
df_C = df[['scene_id', 'text', 'character', 'cCON']]
df_E = df[['scene_id', 'text', 'character', 'cEXT']]
df_O = df[['scene_id', 'text', 'character', 'cOPN']]
df_N = df[['scene_id', 'text', 'character', 'cNEU']]
df_A.to_csv('Friends_A.tsv', sep='\t') 
df_C.to_csv('Friends_C.tsv', sep='\t')
df_E.to_csv('Friends_E.tsv', sep='\t')
df_O.to_csv('Friends_O.tsv', sep='\t')
df_N.to_csv('Friends_N.tsv', sep='\t')


def get_text_role(sent_list, role):
    '''
    Extract the utterances from the given role
    '''
    ans = ""
    for i in sent_list:
        if i[0].split(' ')[0] == role:
            ans = ans + ' ' + i[1]
    return ans


df_A['utterance'] = df_A.apply(lambda r: get_text_role(r['text'], r['character']), axis=1)
df_C['utterance'] = df_C.apply(lambda r: get_text_role(r['text'], r['character']), axis=1)
df_E['utterance'] = df_E.apply(lambda r: get_text_role(r['text'], r['character']), axis=1)
df_O['utterance'] = df_O.apply(lambda r: get_text_role(r['text'], r['character']), axis=1)
df_N['utterance'] = df_N.apply(lambda r: get_text_role(r['text'], r['character']), axis=1)

df_A['labels'] = df_A['cAGR']
df_C['labels'] = df_C['cCON']
df_E['labels'] = df_E['cEXT']
df_O['labels'] = df_O['cOPN']
df_N['labels'] = df_N['cNEU']

<ipython-input-21-1249db4ffca8>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_A['utterance'] = df_A.apply(lambda r: get_text_role(r['text'], r['character']), axis=1)
<ipython-input-21-1249db4ffca8>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_C['utterance'] = df_C.apply(lambda r: get_text_role(r['text'], r['character']), axis=1)
<ipython-input-21-1249db4ffca8>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [20]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import RobertaForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

MAX_LEN = 256
SEED = 0
batch_size = 32

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
model     = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).cuda(0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [40]:
def load_data(df, vad='v'):
    input_ids = [tokenizer.encode(sent, add_special_tokens=True, max_length=MAX_LEN,pad_to_max_length=True) for sent in df['utterance']]
    attention_masks = []
    attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
    
    labels = list(df['labels'])
    train_inputs,test_inputs,train_labels,test_labels = \
        train_test_split(input_ids, labels, random_state=SEED, test_size=0.1, stratify=labels)
    train_masks,test_masks,_,_ = train_test_split(attention_masks,labels,random_state=SEED,test_size=0.1,  stratify=labels)
    
    train_inputs      = torch.tensor(train_inputs)
    test_inputs       = torch.tensor(test_inputs)
    train_labels        = torch.tensor(train_labels)    
    test_labels         = torch.tensor(test_labels)
    train_masks = torch.tensor(train_masks)
    test_masks = torch.tensor(test_masks)


    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    test_data = TensorDataset(test_inputs, test_masks, test_labels)
    test_sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    train_length = len(train_data)
    return train_dataloader, test_dataloader, train_length

train_dataloader, test_dataloader, train_length = load_data(df_E)

/home/zhiyuan/ENTER/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
import numpy as np

# Parameters:
lr = 1e-5
adam_epsilon = 1e-8
epochs = 10

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler


from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, matthews_corrcoef
from tqdm import tqdm, trange,tnrange, tqdm_notebook

## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()

for _ in tnrange(1,epochs+1,desc='Epoch'):
    print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
    # Calculate total loss for this epoch
    batch_loss = 0

    for step, batch in enumerate(train_dataloader):
        # Set our model to training mode (as opposed to evaluation mode)
        model.train()
        
        # Add batch to GPU
        batch = tuple(t.cuda(0) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        
        # Backward pass
        loss.backward()
        
        # Clip the norm of the gradients to 1.0
        # Gradient clipping is not in AdamW anymore
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        
        # Update learning rate schedule
        scheduler.step()

        # Clear the previous accumulated gradients
        optimizer.zero_grad()
        
        # Update tracking variables
        batch_loss += loss.item()

    # Calculate the average loss over the training data.
    avg_train_loss = batch_loss / len(train_dataloader)

    #store the current learning rate
    for param_group in optimizer.param_groups:
        print("\n\tCurrent Learning rate: ",param_group['lr'])
        learning_rate.append(param_group['lr'])
      
    train_loss_set.append(avg_train_loss)
    print(F'\n\tAverage Training loss: {avg_train_loss}')
      
    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0
    
    labels_list = np.array([])
    pred_list = np.array([])

    # Evaluate data for one epoch
    for batch in test_dataloader:
        # Add batch to GPU
        batch = tuple(t.cuda(0) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          logits = model(b_input_ids, attention_mask=b_input_mask)
        
        # Move logits and labels to CPU
        logits = logits[0].to('cpu').numpy()
        label_ids = b_labels.to('cpu').numpy()

        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        
        pred_list = np.append(pred_list, pred_flat)
        labels_list = np.append(labels_list, labels_flat)
        
        df_metrics=pd.DataFrame({'Epoch':epochs,'Actual_class':labels_flat,'Predicted_class':pred_flat})
        
        tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
        tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)
        
        eval_accuracy += tmp_eval_accuracy
        eval_mcc_accuracy += tmp_eval_mcc_accuracy
        nb_eval_steps += 1

    print(classification_report(pred_list, labels_list, digits=4))

<ipython-input-41-7a44a21738ee>:26: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for _ in tnrange(1,epochs+1,desc='Epoch'):


<====================== Epoch 1 ======================>

	Current Learning rate:  9e-06

	Average Training loss: 1.0111525990068913
              precision    recall  f1-score   support

         0.0     0.3636    0.7059    0.4800        17
         1.0     0.7917    0.4750    0.5937        40

    accuracy                         0.5439        57
   macro avg     0.5777    0.5904    0.5369        57
weighted avg     0.6640    0.5439    0.5598        57

<====================== Epoch 2 ======================>

	Current Learning rate:  8.000000000000001e-06

	Average Training loss: 0.6487028673291206
              precision    recall  f1-score   support

         0.0     0.8182    0.7500    0.7826        36
         1.0     0.6250    0.7143    0.6667        21

    accuracy                         0.7368        57
   macro avg     0.7216    0.7321    0.7246        57
weighted avg     0.7470    0.7368    0.7399        57

<====================== Epoch 3 ======================>

	Current 

In [5]:
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
labels_list = np.array([1,2,3])
pred_list = np.array([2,1,3])
print(classification_report(pred_list, labels_list, digits=4, output_dict=True))

{'1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, '3': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1}, 'accuracy': 0.3333333333333333, 'macro avg': {'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1-score': 0.3333333333333333, 'support': 3}, 'weighted avg': {'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1-score': 0.3333333333333333, 'support': 3}}
